In [ ]:
#hide
from utilities.ipynb_docgen import *
from nbdev import *


# B1259 Analysis
> Create a B1259 light curve using Bayesean Blocks  

In [ ]:
#collapse-hide
from wtlike.config import *
from wtlike.bayesian import *
from wtlike.simulation import *
from wtlike.lightcurve import *
from wtlike.loglike import *
from wtlike.cells import *
config = Config()

def bb_overplot(config, lc, bb_fit, ax=None, colors=('black', 'papayawhip', 'blue'), **kwargs):
    fig, ax = plt.subplots(1,1, figsize=(12,4)) if not ax else (ax.figure, ax)
    flux_plot(config, lc, ax=ax, colors=colors, **kwargs)
    flux_plot(config, bb_fit, ax=ax, step=True, colors=colors, **kwargs)
    
def analyze_data(config, source):
    """
    Analyze data from the source
    
    Returns, data a partitioned light curves
    """
    lc = get_lightcurve(config, source)
    cells = get_cells(config, source)
    edges = get_bb_partition(config, lc, LikelihoodFitness, key='bb-Geminga-test') 
    bb_cells = partition_cells(config, cells, edges);
    bb_lc  = fit_cells(config, bb_cells, )
    return lc, bb_lc

def fit_table(lc, expect=1.0):
    """Generate a summary table from a light curve"""
    fits = lc.fit
    flux = fits.apply(lambda f: f.flux)
    errors = fits.apply(lambda f: (round(f.errors[0]-f.flux,3), round(f.errors[1]-f.flux ,3) ) )
    sigma_dev = fits.apply(lambda f: round(f.poiss.sigma_dev(expect),1) )
    df = lc['t tw n'.split()].copy() # maybe fix warnings?
    df.loc[:,'flux'] = flux.values.round(4)
    df.loc[:, 'errors'] = errors.values
    df.loc[:, 'sigma_dev'] = sigma_dev.values
    return df

In [ ]:
config=Config(cachepath='/tmp/cache2')

In [ ]:
#collapse-hide
bb_lc = None
lc = None
def peri(name='PSR_B1259-63', expect=1):
    """
    ## Fit to all data
    
    Combine the likelihoods for all the data, check the fit.
    This defines the base normalization.
    {fig1}
    
    ### The full light curve, showing the BB partitions
    {fig2}
    ### Periastron dates
    Assuming {p}-day orbital period, the MJD and UTC values are:
    
    {utc}
    
    Expand the above about those dates
    
    {fig3}
    """
    global lc, bb_lc
    
    plt.rc('font', size=14)
    source = PointSource(name)
    cells = get_cells(config, source)
    cell = concatenate_cells(cells)
    

    ll = LogLike(cell); 
    #print(ll,'\nFit: ', ll.fit_info())
    fig1, ax1 = plt.subplots(num=1, figsize=(3,2))
    ll.plot(xlim =( 0.5, 1.5), ax=ax1)
   
    with capture_print('Analysis output') as outp:
        lc, bb_lc = analyze_data(config, source)
        
    pd.set_option('display.precision', 3)#, 'display.colheader_justify','left')
    
    df = fit_table(bb_lc, expect=expect)
    df_text = monospace(str(df), 'BB fit table', open=True)
        
    plt.rc('font', size=16)
    fig2, ax2 = plt.subplots(1,1, sharex=True, figsize=(20,4), num=2)
    bb_overplot(config, lc, bb_lc, ax = ax2)
    ax2.text(0.05, 0.85, name,  transform=ax2.transAxes);
    ax2.set(yscale='log')
    fig2.width=600
    
    tp=55546
    p = 1237
    utc = dict((tp+n*p, UTC(tp+n*p)) for n in range(4))
    
    fig3, axx = plt.subplots(3,1, sharex=False, sharey=True, figsize=(12,12), num=3)
    plt.subplots_adjust(hspace=0.3)
    for i, ax in enumerate(axx.flatten()):
        bb_overplot(config, lc, bb_lc, ax=ax, tzero=tp+i*p, xlim=(-30,90), yscale='log', colors=('black','wheat','blue'))
        ax.text(0.02, 0.9, UTC(tp+i*p)[:4], transform=ax.transAxes)
    return locals()
nbdoc(peri)

Cell data for PSR_B1259-63: Restoring from cache with key "cells_PSR_B1259-63"


## Fit to all data

Combine the likelihoods for all the data, check the fit.
This defines the base normalization.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/peri_fig_01.png" title="images/peri_fig_01.png">    <img src="images/peri_fig_01.png" alt="Figure 1 at images/peri_fig_01.png" >   </a> </figure>
</div>


### The full light curve, showing the BB partitions
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/peri_fig_02.png" title="images/peri_fig_02.png">    <img src="images/peri_fig_02.png" alt="Figure 2 at images/peri_fig_02.png" width=600>   </a> </figure>
</div>

### Periastron dates
Assuming 1237-day orbital period, the MJD and UTC values are:

<p style="margin-left: 5%"><samp>{ 55546: '2010-12-16 00:00',
  56783: '2014-05-06 00:00',
  58020: '2017-09-24 00:00',
  59257: '2021-02-12 00:00'}</samp></p>

Expand the above about those dates

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 3">  <a href="images/peri_fig_03.png" title="images/peri_fig_03.png">    <img src="images/peri_fig_03.png" alt="Figure 3 at images/peri_fig_03.png" >   </a> </figure>
</div>



## Generate a light curve summary

Save the csv file [here](images/B1259_lc.csv)

In [ ]:
#hide
lc_out = bb_lc.copy()

lc_out.loc[:,'flux'] = bb_lc.fit.apply(lambda fit: round(fit.flux,3))
errors = bb_lc.fit.apply(lambda fit: np.array(fit.errors).round(3)).values
elow, ehigh = [[x[i] for x in errors] for i in range(2)]
lc_out['low']=elow
lc_out['high']=ehigh
lc_out['t tw n flux low high'.split()].to_csv('images/B1259_lc.csv')

In [ ]:
#collapse-hide
def bb_overplot(config, lc, bb_fit, ax=None,  **kwargs):
    fig, ax = plt.subplots(1,1, figsize=(12,4)) if not ax else (ax.figure, ax)
    flux_plot(config, lc, ax=ax, colors=(('lightblue', 'sandybrown', 'blue')),**kwargs)
    flux_plot(config, bb_fit, ax=ax, step=True, **kwargs)
    
def simulation(config, source, bb_key=None):
    """Create and analyze a simulation for the source
    Returns the simulated, and fit light curves
    """

    lc = get_lightcurve(config, source)
    data_cells = get_cells(config, source)

    #  Get the rate from the data
    cq = data_cells.query('e>0.3')
    T, N = np.sum(cq.tw), np.sum(cq.n)
    sflux=lambda t: N/T

    # simulate, then fit cells to create a simulated light curve 
    sim_cells = simulate_cells(config, source, source_flux=sflux  )
    sim_lc  = fit_cells(config, sim_cells) 

    sim_edges = get_bb_partition(config, sim_lc,  key=bb_key) #'simulated_BB_partition_Geminga') 

    # partion, then fit the cells according to the edges
    sim_bb_cells = partition_cells(config, sim_cells, sim_edges);
    sim_bb_fit  = fit_cells(config, sim_bb_cells, )
    return sim_lc, sim_bb_fit

def analyze_data(config, source):
    """
    Analyze data from the source
    
    Returns, data a partitioned light curves
    """
    lc = get_lightcurve(config, source)
    cells = get_cells(config, source)
    edges = get_bb_partition(config, lc, LikelihoodFitness, key='bb-Geminga-test') 
    bb_cells = partition_cells(config, cells, edges);
    bb_lc  = fit_cells(config, bb_cells, )
    return lc, bb_lc

def fit_table(lc, expect=1.0):
    """Generate a summary table from a light curve"""
    fits = lc.fit
    flux = fits.apply(lambda f: f.flux)
    errors = fits.apply(lambda f: (round(f.errors[0]-f.flux,3), round(f.errors[1]-f.flux ,3) ) )
    sigma_dev = fits.apply(lambda f: round(f.poiss.sigma_dev(expect),1) )
    df = lc['t tw n'.split()].copy() # maybe fix warnings?
    df.loc[:,'flux'] = flux.values.round(4)
    df.loc[:, 'errors'] = errors.values
    df.loc[:, 'sigma_dev'] = sigma_dev.values
    return df